# Making Your Model Learn Addition!
##  Introduction

Given the string "54+7", the model should return a prediction: "61".

In [1]:
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
!pip3 install numpy
!pip3 install keras
!pip3 install tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from termcolor import colored

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

# Generating Data

In [2]:
all_chars='0123456789+'

In [3]:
num_features=len(all_chars)
print('number of features ',num_features)
char_to_index=dict((c,i)for i,c in enumerate(all_chars))
index_to_char=dict((i,c)for i,c in enumerate(all_chars))


number of features  11


In [4]:
def generate_data():
    first=np.random.randint(0,100)
    second=np.random.randint(0,100)
    example=str(first)+'+'+str(second)
    label=str(first+second)
    return example, label
generate_data()

('95+20', '115')

# Create the Model

In [5]:
hidden_units=128
max_time_steps=5

model=Sequential([
    #encoder
    SimpleRNN(hidden_units,input_shape=(None,num_features)),
    RepeatVector(max_time_steps),
    # decoder
    SimpleRNN(hidden_units,return_sequences=True),
    TimeDistributed(Dense(num_features,activation='softmax'))
])
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


# Vectorize and De-Vectorize Data

In [6]:
def vectorize_example(example,label):
    x = np.zeros((max_time_steps,num_features))
    y = np.zeros((max_time_steps,num_features))
    
    
    diff_x = max_time_steps-len(example)
    diff_y = max_time_steps-len(label)
    
    for i,c in enumerate(example):
        x[i+diff_x,char_to_index[c]]=1
    for i in range(diff_x):
        x[i,char_to_index['0']]=1
    for i,c in enumerate(label):
        y[i+diff_y,char_to_index[c]]=1
    for i in range(diff_y):
        y[i,char_to_index['0']]=1
    return x,y
e,l=generate_data()
print(e,l)
x,y=vectorize_example(e,l)
print(x.shape,y.shape)
    
    
        
    
    

34+38 72
(5, 11) (5, 11)


In [7]:
def devectorize_example(example):
    result=[index_to_char[np.argmax(vec)]for i,vec in enumerate(example)]
    return ''.join(result)
devectorize_example(x)


'34+38'

In [8]:
devectorize_example(y)

'00072'

#  Creating dataset 

In [9]:
def create_dataset(num_examples=2000):
    x=np.zeros((num_examples,max_time_steps,num_features))
    y=np.zeros((num_examples,max_time_steps,num_features))
    for i in range(num_examples):
        e,l=generate_data()
        e_v,l_v=vectorize_example(e,l)
        x[i]=e_v
        y[i]=l_v
    return x,y


x,y=create_dataset()
print(x.shape,y.shape)



(2000, 5, 11) (2000, 5, 11)


In [10]:
devectorize_example(x[0])

'05+35'

In [11]:
devectorize_example(y[0])

'00040'

# Training model


In [12]:
l_cb=LambdaCallback(
    on_epoch_end=lambda e,l: print('{:.2f}'.format(l['val_acc']),end='_ '))
es_cb=EarlyStopping(monitor='val_loss',patience=10)
model.fit(x,y,epochs=500,batch_size=256,validation_split=0.2,verbose=False,callbacks=[es_cb,l_cb])


Instructions for updating:
Use tf.cast instead.
0.54_ 0.60_ 0.60_ 0.61_ 0.61_ 0.61_ 0.62_ 0.63_ 0.63_ 0.64_ 0.64_ 0.65_ 0.65_ 0.65_ 0.65_ 0.65_ 0.66_ 0.66_ 0.67_ 0.66_ 0.67_ 0.68_ 0.69_ 0.68_ 0.69_ 0.70_ 0.70_ 0.70_ 0.70_ 0.71_ 0.71_ 0.71_ 0.73_ 0.72_ 0.74_ 0.74_ 0.73_ 0.75_ 0.74_ 0.75_ 0.75_ 0.75_ 0.76_ 0.76_ 0.76_ 0.77_ 0.77_ 0.77_ 0.77_ 0.78_ 0.78_ 0.78_ 0.78_ 0.80_ 0.80_ 0.81_ 0.80_ 0.81_ 0.82_ 0.82_ 0.83_ 0.83_ 0.83_ 0.84_ 0.84_ 0.85_ 0.84_ 0.85_ 0.85_ 0.85_ 0.85_ 0.85_ 0.85_ 0.86_ 0.86_ 0.87_ 0.87_ 0.87_ 0.87_ 0.87_ 0.88_ 0.88_ 0.88_ 0.88_ 0.88_ 0.89_ 0.89_ 0.89_ 0.89_ 0.89_ 0.89_ 0.90_ 0.89_ 0.90_ 0.90_ 0.90_ 0.90_ 0.90_ 0.90_ 0.90_ 0.91_ 0.91_ 0.90_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.91_ 0.92_ 0.92_ 0.91_ 0.92_ 0.92_ 0.91_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.92_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93_ 0.93

In [13]:
x_test,y_test=create_dataset(10)
preds=model.predict(x_test)

for i,pred in enumerate(preds):
    y=devectorize_example(y_test[i])
    y_hat=devectorize_example(pred)
    col='green'
    if y!=y_hat:
        col='red'
        
    out='Input'+devectorize_example(x_test[i])+'Out :'+y+ ' pred: '+y_hat
    print(colored(out,col))
    
    
    

Input76+99Out :00175 pred: 00175
Input021+6Out :00027 pred: 00030
Input89+85Out :00174 pred: 00164
Input21+28Out :00049 pred: 00049
Input03+89Out :00092 pred: 00092
Input73+39Out :00112 pred: 00112
Input51+46Out :00097 pred: 00097
Input86+26Out :00112 pred: 00112
Input56+62Out :00118 pred: 00118
Input59+13Out :00072 pred: 00072
